# Plotting a Time Series of HMDA Filers by Category

## Import Python Libraries

In [28]:
import psycopg2 #Imports the Psycopg2 library
import pandas as pd #Imports the Pandas library and renames it "pd"
import matplotlib.pyplot as plt #imports the Matplot library and renames it "plt"
import numpy as np

### Connect to the Database
The connection to the database, which was demonstrated in the [previous example](https://github.com/cfpb/HMDA_Data_Science_Kit/blob/master/analysis_examples/1.%20Plotting%20a%20Time%20Series%20of%20HMDA%20Filers%20from%202004-2017.ipynb), will use a locally hosted database and the hmda database created during initial setup. Please see Analysis Example 1 for further details on connecting using a locally hosted database. 

In [29]:
#Establish connection parameters
#If you have established a username and password, change user and password below to your own username and password.
connection_params = {"user":"postgres", 
                     "password":"helloworld", 
                     "dbname":"hmda", 
                     "host":"localhost",
                     "port" : "5433"}

In [30]:
def connect(params):
    """
    This function accepts a dictionary of connection parameters that must include:
    - user: the username to be used for the database session
    - password: the user's password
    - dbname: the name of the database for connection
    - host: the host location of the database
    """
    #attempt a connection with the supplied parameters
    try:
        conn = psycopg2.connect(**params)
        print("I'm connected") #print a success message
        return conn.cursor() #return a cursor object
    except psycopg2.Error as e: 
        print("I am unable to connect to the database: ", e) #print a fail message and the error, if any

In [31]:
#Test the connection function, if everything is correct, it will print "I'm connected."
cur = connect(params=connection_params)
#Close the cursor. This is important as open cursors can interfere with updates to data tables.
cur.close()

#When using Jupyter, it is best to open and close the cursor in the same code cell. 
#If there are coding errors that interrupt the execution, the cursor will need to be reestablished. 


I'm connected


### Variabalizing a SQL Command String for Filtering
As demonstrated in the previous example, Python strings can contain markers which enable substitution of values. This allows use of the .format() command to change the table reference for the SQL query. The string below selects the activity year and the filer count, variabalizing the year.   

In [32]:
sql_command = """SELECT 
                    activity_year, 
                    COUNT (*) AS filer_count
                 FROM 
                    hmda_public.ts_{year} 
                 GROUP BY 
                    activity_year;"""

A SQL command string may be modified to select a count of filers by a given category. The command string below not only variabalizes the year of the file to be selected but also creates an extention to the query that may be modified by the user.  

In [33]:
sql_base = """SELECT msa,
           count(case when lar_{year}.loan_purpose = '1' then 1 else null end) as prps_prch,
           count(case when lar_{year}.loan_purpose = '2' then 1 else null end) as prps_impr,
           count(case when lar_{year}.loan_purpose = '3' then 1 else null end) as prps_refi
            FROM lar_{year} {extention}
            ;"""

In [34]:
extention = " GROUP BY msa"
 

### Selecting the Number of Filers by a Particular State
The code above may be placed in another sql file, which may be called by the the time series function defined in the previous example. The code below may be used to select for the number of filers who have their headquarters in New York. Respondent_state refers to the headquarters location of the institution and does not necessarily reflect lending patterns in that geography.

In [35]:
cur = connect(connection_params) #Establishes cursor object and connect to the database
year = 2016

#Provides the year of the file 

#Sets the extention variable so that it selects filers in the state of New York
print(sql_base.format(year=year, extention=extention))
 
#Executes the query text against the database, formatting for the year and the extention
cur.execute(sql_base.format(year=year, extention=extention))
results = cur.fetchall() #Returns the query results.
#print(results)
#print(cur.description)
#Converts the results_list into a Pandas dataframe with names pulled from the SQL query.
results_df = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])
cur.close() #Closes the connection and remove the cursor object.
results_df #Shows the top 5 rows of the dataframe

I'm connected
SELECT msa,
           count(case when lar_2016.loan_purpose = '1' then 1 else null end) as prps_prch,
           count(case when lar_2016.loan_purpose = '2' then 1 else null end) as prps_impr,
           count(case when lar_2016.loan_purpose = '3' then 1 else null end) as prps_refi
            FROM lar_2016  GROUP BY msa
            ;


,msa,prps_prch,prps_impr,prps_refi
0,10180,4110,590,2382
1,10380,1323,1436,1002
2,10420,14339,1862,14260
3,10500,2017,574,2670
4,10540,3336,309,3397
5,10580,16645,2829,12261
6,10740,19535,2399,25117
7,10780,2888,449,2735
8,10900,16568,2733,17494
9,11020,1577,676,1964


In [36]:
sql_base = """SELECT CONCAT(lar_{year}.state_code,lar_{year}.county_code) AS fips,
           count(case when lar_{year}.loan_purpose = '1' then 1 else null end) as prps_prch,
           count(case when lar_{year}.loan_purpose = '2' then 1 else null end) as prps_impr,
           count(case when lar_{year}.loan_purpose = '3' then 1 else null end) as prps_refi
            FROM lar_{year} {extention}
            ;"""

In [37]:
extention = " GROUP BY state_code,county_code Limit 500"
 

In [38]:
cur = connect(connection_params) #Establishes cursor object and connect to the database
year = 2016

#Provides the year of the file 

#Sets the extention variable so that it selects filers in the state of New York

 
#Executes the query text against the database, formatting for the year and the extention
cur.execute(sql_base.format(year=year, extention=extention))
results = cur.fetchall() #Returns the query results.
#print(results)

#Converts the results_list into a Pandas dataframe with names pulled from the SQL query.
results_df = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])
cur.close() #Closes the connection and remove the cursor object.
results_df.head() #Shows the top 5 rows of the dataframe

I'm connected


,fips,prps_prch,prps_impr,prps_refi
0,01001,1510,167,1517
1,01003,7977,752,6310
2,01005,268,44,280
3,01007,290,85,297
4,01009,1271,158,1180


In [42]:
results_df = results_df[ ~ results_df.fips.str.contains('NA', case=False,regex=True)]


In [43]:
results_df.head()

,fips,prps_prch,prps_impr,prps_refi
0,01001,1510,167,1517
1,01003,7977,752,6310
2,01005,268,44,280
3,01007,290,85,297
4,01009,1271,158,1180


In [45]:
df3= results_df[['fips', 'prps_refi']]

In [46]:
import plotly
plotly.tools.set_credentials_file(username='levitan.matt', api_key='ezbwoqyVCTSUpcY0pcLc')

In [47]:
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np

In [48]:
results_df = results_df[ ~ results_df.fips.str.contains('NA', case=False,regex=True)]
fips = results_df.fips
values = results_df.prps_refi

bins =list(np.mgrid[min(values):max(values):10j])

fig =ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=bins)
py.plot(fig, filename='choropleth of us counties - refi LAR')

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




'https://plot.ly/~levitan.matt/8'

In [27]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

trace4 = go.Scatter(x=[4000, 5000, 6000], y=[7000, 8000, 9000])
trace1 = go.Scatter(x=[1, 2, 3], y=[4, 5, 6])
trace2 = go.Scatter(x=[20, 30, 40], y=[50, 60, 70])
trace3 = go.Scatter(x=[300, 400, 500], y=[600, 700, 800])

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Plot 1', 'Plot 2',
                                                          'Plot 3', 'Plot 4'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(height=600, width=600, title='Multiple Subplots' +
                                                  ' with Titles')

py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

